In [ ]:
## mock dipole and check sanity 
#This code is written by Rahul Kothari and Prabhakar Tiwari
import multiprocessing
from multiprocessing import Manager
from joblib import Parallel, delayed
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import math
from scipy.stats import norm 
from scipy.stats import poisson
import fitsio
import sys
from numpy.linalg import norm
import scipy

In [ ]:
def ma_to_array(m):
    """Converts a masked array or a list of masked arrays to filled numpy arrays

    Parameters
    ----------
    m : a map (may be a sequence of maps)

    Returns
    -------
    m : filled map or tuple of filled maps

    Examples
    --------
    >>> import healpy as hp
    >>> m = hp.ma(np.array([2., 2., 3, 4, 5, 0, 0, 0, 0, 0, 0, 0]))
    >>> m.mask = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.bool_)
    >>> print(m.data[1]) # data is not affected by mask
    2.0
    >>> print(m[1]) # shows that the value is masked
    --
    >>> print(ma_to_array(m)[1]) # filled array, masked values replace by UNSEEN
    -1.6375e+30
    """
    try:
        return m.filled()
    except AttributeError:
        try:
            return np.array([mm.filled() for mm in m])
        except AttributeError:
            pass
    return m

def monopole(MonoInfo, nside):
    return MonoInfo*np.ones(hp.nside2npix(nside))

def dipole(DipoInfo, nside):
    Dx, Dy, Dz = DipoInfo
    ipix = np.arange(0,hp.nside2npix(nside))
    x, y, z = hp.pix2vec(nside,ipix)
    return Dx*x + Dy*y + Dz*z

def quadrupole(QuadInfo, nside):
    Qxy, Qxz, Qyz, QzSq, QxSqMySq = QuadInfo
    ipix = np.arange(0,hp.nside2npix(nside))
    x, y, z = hp.pix2vec(nside,ipix)
    return Qxy*x*y + Qyz*y*z + Qxz*x*z + QzSq*(2.0*z**2-y**2-x**2) + QxSqMySq*(x**2-y**2)

def octupole(OctoInfo, nside):
    O1, O2, O3, O4, O5, O6, O7 = OctoInfo
    ipix = np.arange(0,hp.nside2npix(nside))
    x, y, z = hp.pix2vec(nside,ipix)
    rSq = x**2 + y**2 + z**2
    term1 = O1*y*(3.0*x**2 - y**2) + O2*x*y*z + O3*y*(5.0*z**2 - rSq) + O4*z*(5.0*z**2-3.0*rSq)
    term2 = O5*x*(5.0*z**2 - rSq) + O6*z*(x**2-y**2) + O7*x*(x**2 - 3*y**2)
    return term1 + term2

def fit_quadrupole(m, nest=False, bad=hp.UNSEEN, gal_cut=0):
    """Fit a quadrupole, dipole and a monopole to the map, excluding bad pixels.

    Parameters
    ----------
    m : float, array-like
      the map to which a dipole is fitted and subtracted, accepts masked maps
    nest : bool
      if ``False`` m is assumed in RING scheme, otherwise map is NESTED
    bad : float
      bad values of pixel, default to :const:`UNSEEN`.
    gal_cut : float [degrees]
      pixels at latitude in [-gal_cut;+gal_cut] degrees are not taken into account

    Returns
    -------
    res : tuple of length 2
      the monopole value in res[0] and the dipole vector (as array) in res[1]

    See Also
    --------
    remove_dipole, fit_monopole, remove_monopole
    """
    m = ma_to_array(m)
    m = np.asarray(m)
    npix = m.size
    nside = hp.npix2nside(npix)
    if nside > 128:
        bunchsize = npix // 24
    else:
        bunchsize = npix
    aa = np.zeros((9, 9), dtype=np.float64)
    v = np.zeros(9, dtype=np.float64)
    for ibunch in range(npix // bunchsize):
        ipix = np.arange(ibunch * bunchsize, (ibunch + 1) * bunchsize)
        ipix = ipix[(m.flat[ipix] != bad) & (np.isfinite(m.flat[ipix]))]
        x, y, z = hp.pix2vec(nside, ipix, nest)
        if gal_cut > 0:
            w = np.abs(z) >= np.sin(gal_cut * np.pi / 180)
            ipix = ipix[w]
            x = x[w]
            y = y[w]
            z = z[w]
            del w
        
        row1 = np.array([np.ones(ipix.size), x, y, z, x*y, x*z, y*z, 2.0*z**2-x**2-y**2, x**2-y**2])
        col1 = row1
    
        for i in range(9):
            aa[i,:] += np.sum(row1*col1[i],axis=1)
        
        v += np.sum(m.flat[ipix]*col1,axis=1)
    
    res = np.dot(np.linalg.inv(aa), v)
    mono = res[0]
    dipole = res[1:4]
    quadru = res[4:9]
    return mono, dipole, quadru

def fit_octupole(m, nest=False, bad=hp.UNSEEN, gal_cut=0):
    """Fit a quadrupole, dipole and a monopole to the map, excluding bad pixels.

    Parameters
    ----------
    m : float, array-like
      the map to which a dipole is fitted and subtracted, accepts masked maps
    nest : bool
      if ``False`` m is assumed in RING scheme, otherwise map is NESTED
    bad : float
      bad values of pixel, default to :const:`UNSEEN`.
    gal_cut : float [degrees]
      pixels at latitude in [-gal_cut;+gal_cut] degrees are not taken into account

    Returns
    -------
    res : tuple of length 2
      the monopole value in res[0] and the dipole vector (as array) in res[1]

    See Also
    --------
    remove_dipole, fit_monopole, remove_monopole
    """
    m = ma_to_array(m)
    m = np.asarray(m)
    npix = m.size
    nside = hp.npix2nside(npix)
    if nside > 128:
        bunchsize = npix // 24
    else:
        bunchsize = npix
    aa = np.zeros((16, 16), dtype=np.float64)
    v = np.zeros(16, dtype=np.float64)
    for ibunch in range(npix // bunchsize):
        ipix = np.arange(ibunch * bunchsize, (ibunch + 1) * bunchsize)
        ipix = ipix[(m.flat[ipix] != bad) & (np.isfinite(m.flat[ipix]))]
        x, y, z = hp.pix2vec(nside, ipix, nest)
        if gal_cut > 0:
            w = np.abs(z) >= np.sin(gal_cut * np.pi / 180)
            ipix = ipix[w]
            x = x[w]
            y = y[w]
            z = z[w]
            del w
        rSq = x**2 + y**2 + z**2 
        
        row1 = np.array([np.ones(ipix.size), x, y, z, x*y, x*z, y*z, 2.0*z**2-x**2-y**2, x**2-y**2, 
                        y*(3.0*x**2 - y**2), x*y*z, y*(5.0*z**2 - rSq), z*(5.0*z**2-3.0*rSq), x*(5.0*z**2 - rSq),
                        z*(x**2-y**2), x*(x**2 - 3*y**2)])
        col1 = row1
        
        for i in range(16):
            aa[i,:] += np.sum(row1*col1[i],axis=1)
        
        v += np.sum(m.flat[ipix]*col1,axis=1)
    
    res = np.dot(np.linalg.inv(aa), v)
    mono = res[0]
    dipole = res[1:4]
    quadru = res[4:9]
    octu = res[9:16]
    return mono, dipole, quadru, octu

def fit_plot_gaus(data, prior, name):
    ftSz = 12
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    mu, std = scipy.stats.norm.fit(data)
    plt.figure(figsize=(5,5))
    plt.hist(data, bins=50, density=True, histtype ="bar", color=(0.75,0.75,0.75), edgecolor = "k")
    # Plot the PDF.
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = scipy.stats.norm.pdf(x, mu, std)
    plt.plot(x, p, 'r', linewidth=2)
    plt.axvline(x = prior,c='r',ls='--')
#     print(mu,std)
    diction =  {'Mono' : '$N_0$',
             'Dx' :'$\mathcal{D}_x$',
             'Dy' :'$\mathcal{D}_y$',
             'Dz' : '$\mathcal{D}_z$',
             'Qxy' : '$\mathcal{Q}_{xy}$',
             'Qxz' :'$\mathcal{Q}_{xz}$',
             'Qyz' :'$\mathcal{Q}_{yx}$',
             'QzSq' :'$\mathcal{Q}_{z^2}$',
             'QxSqMySq':'$\mathcal{Q}_{x^2-y^2}$'
            }
    title = r""+str(diction[name])+" ($\mu=$ %6.4f, $\mathrm{prior}=$ %6.4f, $\sigma=$ %6.4f)"%(mu, prior, std)
    plt.title(title,fontsize=14)
    plt.xticks(fontsize=ftSz)
    plt.yticks(fontsize=ftSz)
    plt.tight_layout()
    plt.savefig(name+'_N.pdf')

In [ ]:
if __name__ == '__main__': # needed for  multiprocessing
    def mock_quadrupole(mock, seeds, map_d, sigma, mask, Mono, DipX, DipY, DipZ, QuaXY, QuaXZ, QuaYZ, QuaZsq, QuaxSqMySq):
        NPIX=len(map_d)
        m_test=np.zeros(NPIX)
        np.random.seed(seed=seeds[mock])
        for i in range(NPIX):
            if mask[i]<1:   ##data below dec<15 SKA1; dec<30 SKA2
                m_test[i]=hp.UNSEEN
            else:
                m_test[i]=np.random.normal(map_d[i], sigma[i])
            #-------
        pars = fit_quadrupole(m_test, nest=False, bad=hp.UNSEEN, gal_cut=0) # assumes masked pixels set to hp.UNSEEN
        mono = pars[0]
        Dx, Dy, Dz = pars[1][:]
        Qxy, Qxz, Qyz, QzSq, QxSqMySq = pars[2][:]
        Mono.append(mono)
        DipX.append(Dx)
        DipY.append(Dy)
        DipZ.append(Dz)
        QuaXY.append(Qxy)
        QuaXZ.append(Qxz)
        QuaYZ.append(Qyz)
        QuaZsq.append(QzSq)
        QuaxSqMySq.append(QxSqMySq)
    
    #-------
    m_agn = hp.read_map('data/Brightness_countNside64.fits', field=0, dtype=float, nest=False)
    sigma = hp.read_map('data/Brightness_countNside64.fits', field=1, dtype=float, nest=False)
    mask = hp.read_map('data/Brightness_countNside64.fits', field=2, dtype=float, nest=False)

    NPIX=len(m_agn)
    NSIDE=hp.npix2nside(NPIX)
    n_pix=0
    n_agns=0
    for pix in range(NPIX):
        if mask[pix]>0:
            n_pix+=1
            n_agns+=m_agn[pix]
    fsky=n_pix/NPIX
    
    pars = fit_quadrupole(m_agn, nest=False, bad=hp.UNSEEN, gal_cut=0) # assumes masked pixels set to hp.UNSEEN
    print("n_bar (count/pixel)= %7.5f(fit)  %7.5f"%(pars[0],n_agns/n_pix))
    mono = pars[0]
    Dx, Dy, Dz = pars[1][:]
    Qxy, Qxz, Qyz, QzSq, QxSqMySq = pars[2][:]
    ## caculate error bars and method calibration
    # make a map with dipole and quadrupole signal
    map_d = mono*np.ones(NPIX) + quadrupole([Qxy, Qxz, Qyz, QzSq, QxSqMySq],NSIDE) + dipole([Dx,Dy,Dz],NSIDE) 
    map_mock=np.zeros(NPIX)
    for i in range(NPIX):
        if mask[i]<1:
            map_d[i]=hp.UNSEEN

    # run parallel 
    manager = Manager()
    Mono = manager.list()
    DipX = manager.list()
    DipY = manager.list()
    DipZ = manager.list()
    QuaXY = manager.list()
    QuaXZ = manager.list()
    QuaYZ = manager.list()
    QuaZsq = manager.list()
    QuaxSqMySq = manager.list()
    
    nn=1000 # no. of mocks 
    
    seeds=np.random.randint(999999999, size=nn)
    num_cores = multiprocessing.cpu_count()
    processed_list = Parallel(n_jobs=num_cores)(delayed(mock_quadrupole)(i,seeds,map_d,sigma,mask,Mono,DipX,DipY,DipZ,QuaXY, QuaXZ, QuaYZ, QuaZsq, QuaxSqMySq) for i in range(nn))
    
#         print()
#     fout=open("dipole_mocks.dat","w")
#     for d1,d2,d3,q1,q2,q3,q4,q5 in zip(DipX,DipY,DipZ,QuaXY, QuaXZ, QuaYZ, QuaZsq, QuaxSqMySq):
#         print(round(d1,6), round(d2,6), round(d3,6), round(q1,6), round(q2,6), round(q3,6), round(q4,6), round(q5,6))
#         fout.write("'{0}' '{1}' '{2}' '{3}' '{4}' '{5}' '{6}' '{7}'".format(d1,d2,d3,q1,q2,q3,q4,q5))
#     fout.close()

In [ ]:
fit_plot_gaus(Mono, mono, 'Mono')
fit_plot_gaus(DipX, Dx, 'Dx')
fit_plot_gaus(DipY, Dy, 'Dy')
fit_plot_gaus(DipZ, Dz, 'Dz')
fit_plot_gaus(QuaXY, Qxy, 'Qxy')
fit_plot_gaus(QuaXZ, Qxz, 'Qxz')
fit_plot_gaus(QuaYZ, Qyz, 'Qyz')
fit_plot_gaus(QuaZsq, QzSq, 'QzSq')
fit_plot_gaus(QuaxSqMySq, QxSqMySq, 'QxSqMySq')